### **Цель семинара**

Построить полноценного RAG-бота на русском корпусе XQuAD-ru: начать с простого векторного поиска (FAISS + эмбеддинги) и добавить режим с кросс-энкодерным reranker’ом, чтобы увидеть, как меняется качество ответов. Итогом станет чат-интерфейс в Colab (Gradio), которому можно задавать вопросы

### **Что мы изучим:**
1. Архитектуру RAG систем и компоненты
2. Стратегии chunking и preprocessing документов
3. Эмбеддинги и индекс: paraphrase-multilingual-MiniLM-L12-v2 (384-мерные векторы) + FAISS
4. Reranking
5. Интеграцию с LangChain

-----

### **0. Установка и импорт библиотек**

Устанавливаем всё необходимое для построения RAG системы.

In [ ]:
!pip -q install -U langchain langchain-community langchain-text-splitters langchain-huggingface datasets faiss-cpu sentence-transformers gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.6/486.6 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
pylibcudf-cu12

# 1. Импорты и конфиги RAG

In [ ]:
import os, random, numpy as np
from datasets import load_dataset

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_community.vectorstores import FAISS

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

SEED = 42
random.seed(SEED); np.random.seed(SEED)

N_DOCS_MAX = 1000
CHUNK_SIZE = 500
CHUNK_OVERLAP = 100
K_RETRIEVE = 8
TOP_N_RERANK = 4
EMBEDDING_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
RERANKER_MODEL = "BAAI/bge-reranker-v2-m3"


# 3. Загрузка датасета XQuAD-ru

In [ ]:
ds = load_dataset("google/xquad", "xquad.ru")["validation"]
seen, contexts = set(), []
for ex in ds:
    ctx = ex["context"]
    if ctx not in seen:
        seen.add(ctx)
        contexts.append(ctx)
contexts = contexts[:N_DOCS_MAX]
len(contexts), contexts[0][:160]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

xquad.ru/validation-00000-of-00001.parqu(…):   0%|          | 0.00/322k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/1190 [00:00<?, ? examples/s]

(240,
 '\ufeffЗащита Пэнтерс уступила всего 308 очков, заняв шестое место в лиге, а также лидировала в НФЛ по перехватам с 24 и похвасталась четырьмя попаданиями в Пробоул. ')

# Чанкинг
Иерархический символьный чанкинг с размером CHUNK_SIZE из конфига в начале ноутбука и с перекрытием

In [ ]:
from langchain_core.documents import Document
splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
docs = splitter.split_documents([Document(page_content=c) for c in contexts])
len(docs), docs[0].page_content[:160]


(575,
 '\ufeffЗащита Пэнтерс уступила всего 308 очков, заняв шестое место в лиге, а также лидировала в НФЛ по перехватам с 24 и похвасталась четырьмя попаданиями в Пробоул. ')

# 5. Векторизация и FAISS

In [ ]:
emb = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL, encode_kwargs={"normalize_embeddings": True})
vs = FAISS.from_documents(docs, emb)
retriever = vs.as_retriever(search_kwargs={"k": K_RETRIEVE})
print("VectorStore готов:", len(docs), "чанков")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

VectorStore готов: 575 чанков


# 6. Reranker

In [ ]:
cross_encoder = HuggingFaceCrossEncoder(model_name=RERANKER_MODEL)
compressor = CrossEncoderReranker(model=cross_encoder, top_n=TOP_N_RERANK)
compression_retriever = ContextualCompressionRetriever(base_retriever=retriever, base_compressor=compressor)
print("Reranker готов.")


config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Reranker готов.


# 7. LLM модель

In [ ]:
def build_local_llm():
    model_id = "Qwen/Qwen2.5-0.5B-Instruct"
    tok = AutoTokenizer.from_pretrained(model_id)
    mdl = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto")
    gen = pipeline(
        "text-generation",
        model=mdl,
        tokenizer=tok,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.2,
        top_p=0.95,
        repetition_penalty=1.05,
    )
    return HuggingFacePipeline(pipeline=gen)

llm = build_local_llm()

SYSTEM_PROMPT = (
    "Ты — русскоязычный ассистент и отвечаешь кратко и по делу. "
    "Используй только предоставленный контекст. "
    "Если ответа нет в контексте — честно скажи, что не нашёл ответа."
)

prompt = ChatPromptTemplate.from_template(
    "СИСТЕМА: {system}\nКонтекст:\n{context}\n\nВопрос: {question}\nКраткий ответ (на русском), ссылаясь на факты из контекста:"
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cuda:0


# 8. Варианты пайплайнов с реранкером и без

In [ ]:
# Проверим базовую модель
rag_vanilla = (
    {"context": lambda x: "",  # Передаем пустой контекст
     "question": RunnablePassthrough(),
     "system": lambda _: SYSTEM_PROMPT}
    | prompt | llm | StrOutputParser()
)

print("Чистый LLM (БЕЗ RAG):\n", rag_vanilla.invoke("Кого побил Бронкос в 2015 году, чтобы победить в своем дивизионе?")[:])

In [ ]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

rag_plain = (
    {"context": retriever | RunnableLambda(format_docs),
     "question": RunnablePassthrough(),
     "system": lambda _: SYSTEM_PROMPT}
    | prompt | llm | StrOutputParser()
)

rag_rerank = (
    {"context": compression_retriever | RunnableLambda(format_docs),
     "question": RunnablePassthrough(),
     "system": lambda _: SYSTEM_PROMPT}
    | prompt | llm | StrOutputParser()
)

print("BASELINE:\n", rag_plain.invoke("Кого побил Бронкос в 2015 году, чтобы победить в своем дивизионе?")[:])
print("\nС РЕРАНКЕРОМ:\n", rag_rerank.invoke("Кого побил Бронкос в 2015 году, чтобы победить в своем дивизионе?")[:])


BASELINE:
 Human: СИСТЕМА: Ты — русскоязычный ассистент и отвечаешь кратко и по делу. Используй только предоставленный контекст. Если ответа нет в контексте — честно скажи, что не нашёл ответа.
Контекст:
заливе Салливана в Порт-Филиппе. Поселение насчитывало 40

WITH RERANKER:
 Human: СИСТЕМА: Ты — русскоязычный ассистент и отвечаешь кратко и по делу. Используй только предоставленный контекст. Если ответа нет в контексте — честно скажи, что не нашёл ответа.
Контекст:
Обладая размерами от 1 миллиметра (0,039 дюйма) до 1,5 ме


# 9. Создадим UI интерфейс чат-бота прямо в Colab при помощи Gradio

In [ ]:
# Gradio чат с переключателем reranker
import gradio as gr

def chat_fn(message, history, use_reranker: bool):
    chain = rag_rerank if use_reranker else rag_plain
    try:
        return chain.invoke(message)
    except Exception as e:
        return f"Ошибка: {e}"

with gr.Blocks() as demo:
    gr.Markdown("## RAG-бот (RU, XQuAD) — LangChain + FAISS + (опц.) Cross-Encoder Reranker")
    use_reranker = gr.Checkbox(label="Использовать Reranker (BAAI/bge-reranker-v2-m3)", value=False)

    gr.ChatInterface(
        fn=chat_fn,
        additional_inputs=[use_reranker],  # третий аргумент в chat_fn
        title="RAG-бот по XQuAD-ru",
        # При additional_inputs каждый пример — это [сообщение, значение чекбокса]
        examples=[
            ["Кого побил Бронкос в 2015 году, чтобы победить в своем дивизионе?", False],
            ["Когда была основана первая Варшавская фондовая биржа?", False],
            ["Кем был граф Мелфи?", False],
            # пример сразу с реранкером:
            ["Кого побил Бронкос в 2015 году, чтобы победить в своем дивизионе?", True],
        ],
    )

demo.launch()



/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f49cdf59de33f61680.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Дополнительные ресурсы

**Papers:**
- [RAG: Retrieval-Augmented Generation](https://arxiv.org/abs/2005.11401)
- [REALM: Retrieval-Augmented Language Model Pre-Training](https://arxiv.org/abs/2002.08909)
- [Dense Passage Retrieval](https://arxiv.org/abs/2004.04906)

**Фреймворки:**
- [LangChain](https://github.com/langchain-ai/langchain) - Строим LLM приложения при помощи LangChain
- [LlamaIndex](https://github.com/jerryjliu/llama_index) - Фреймворк для работы с LLM
- [Haystack](https://github.com/deepset-ai/haystack) - фреймворк от компании deepset

**Векторные БД:**
- [Qdrant](https://qdrant.tech/)
- [Weaviate](https://weaviate.io/)
- [Pinecone](https://www.pinecone.io/)
- [ChromaDB](https://www.trychroma.com/)

**Фреймворки для тестов RAG систем:**
- [RAGAS](https://github.com/explodinggradients/ragas)
- [ARES](https://github.com/stanford-futuredata/ARES)